### Sentence Transformers

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import nltk
# nltk.download("punkt")
import warnings
warnings.filterwarnings("ignore")

In [3]:
# pip install transformers

In [3]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelWithLMHead, BertTokenizer,BertModel
import re
from nltk.tokenize import sent_tokenize, word_tokenize
# importing all necessary modules
from gensim.models import Word2Vec
import gensim
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

In [4]:
import calendar
cal=list(calendar.month_name)
for i in list(calendar.month_name[1:]):
    cal.append(i[:3])

In [5]:
df=pd.read_excel("July_August_Sepetember_Oct_samp_comments.xlsx")

In [6]:
df

,Unnamed: 0,V_COMMENTS,N_PARTY_WATCHLIST_MATCH_ID,Counts
0,30016,Mismatch ID number_NRIC,"[6147381, 6003260, 5488573, 6013209, 6007897, ...",20859
1,25242,Mismatch Full Legal Name – Name Does Not Sound...,"[6081737, 4832938, 7491187, 6227658, 5401157, ...",13039
2,30017,Mismatch ID number_NRIC\n,"[5129297, 5086069, 6260885, 5277278, 4821963, ...",8666
3,26176,Mismatch Full Legal Name_Name Does Not Sound S...,"[6190699, 5155337, 6121364, 5155334, 5155339, ...",6680
4,20636,Mismatch Date of Birth (DOB),"[4801383, 4887727, 5254984, 4806610, 6031120, ...",6288
...,...,...,...,...
91980,33158,Mismatch Occupation - Customer work as AGRICUL...,[4845599],1
91981,33157,Mismatch Occupation - Customer work as AGRICUL...,[6149893],1
91982,33156,Mismatch Occupation - Customer work as AGRICUL...,[6135720],1
91983,33155,Mismatch Occupation - Customer work as AGRICUL...,[7502691],1


In [7]:
df.drop(["Unnamed: 0",],axis=1,inplace=True)
df.dropna(inplace=True)

In [12]:
# sequence=df[df["V_COMMENTS"].apply(lambda x: len(x)>100)]["V_COMMENTS"].loc[221569]

In [8]:
df_features=pd.read_csv("Topic_Feature_List_Agg_brd_labs.csv")

In [9]:
# df.drop(["id_2_feature","TFIDF_Feature","Topic_tfidf_lda","All_Topic_tfidf_Probabilities","TF_tfidf_Topic_No"," TF IDF Topic words"],axis=1,inplace=True)

In [10]:
df_features

,S.no,Standardized_Anchors_BRD_LABS,BRD_FLAG,Categories,Category_labs
0,1,Word count difference,Y,Full Legal Name - Word Count,NaN
1,2,Name structure difference,Y,Full Legal Name - Name Structure,NaN
2,3,Name does not sound similiar,Y,Full Legal Name - Name Does Not Sound Similar,NaN
3,4,Native name charachter difference,Y,Full Legal Name - Native Name,Name
4,5,NaN,Y,Category (Vessel vs. Individual vs. Entity),NaN
...,...,...,...,...,...
430,431,Customer - Transaction - Number of transactions,NaN,Account & transactions,NaN
431,432,Customer - Transaction - Amount,NaN,Account & transactions,NaN
432,433,Customer - Transaction - Online / offline,NaN,Account & transactions,NaN
433,434,Customer - Joint Account - Flag,NaN,Account & transactions,NaN


In [11]:
df_features.rename(columns={"Standardized_Anchors_BRD_LABS":"Anchors"},inplace=True)

In [12]:
df_features.dropna(subset=["Anchors"],inplace=True)

In [13]:
df_features

,S.no,Anchors,BRD_FLAG,Categories,Category_labs
0,1,Word count difference,Y,Full Legal Name - Word Count,NaN
1,2,Name structure difference,Y,Full Legal Name - Name Structure,NaN
2,3,Name does not sound similiar,Y,Full Legal Name - Name Does Not Sound Similar,NaN
3,4,Native name charachter difference,Y,Full Legal Name - Native Name,Name
5,6,Date of birth (dob),Y,Date of Birth (DOB),DOB
...,...,...,...,...,...
430,431,Customer - Transaction - Number of transactions,NaN,Account & transactions,NaN
431,432,Customer - Transaction - Amount,NaN,Account & transactions,NaN
432,433,Customer - Transaction - Online / offline,NaN,Account & transactions,NaN
433,434,Customer - Joint Account - Flag,NaN,Account & transactions,NaN


In [17]:
# df_features=df_features.loc[:,["Topic No","Anchor - Topic","Anchor existing list","Presence"]]
df_features["Anchor_2"]=df_features["Anchors"].apply(lambda x: re.sub("\s*-\s*|\s*/\s*|,|\(|\)"," ",x))

In [18]:
df_features["Anchor_2"]=df_features["Anchor_2"].apply(lambda x: re.sub("match|mismatch","",x))

In [19]:
df_features["Anchor_2"].shape

(428,)

In [20]:
df_features["Anchor_2"]=df_features["Anchor_2"].str.lower()

In [21]:
patt_10=re.compile("(match|mismatch|false hit|true hit|material|non-material|non material)",re.IGNORECASE)

df["Match_Mismatch"]=df["V_COMMENTS"].apply(lambda x: re.findall(patt_10,x))

# patt_10=re.compile("(match|mismatch)",re.IGNORECASE)
# patt_11=re.compile("(false hit|true hit)",re.IGNORECASE)
# df["Match_Mismatch"]=df["V_COMMENTS"].str.extract(patt_10)
# df["True_False_hit"]=df["V_COMMENTS"].str.extract(patt_11)

df["V_COMMENTS"].loc[29]

'mismatch ID number  - NRIC'

In [22]:
import ast
# df["Tokens"]=df["Tokens"].apply(lambda x: ast.literal_eval(x))

# Pre-processing on Data

### Step -1 removing noisy data

In [23]:
import calendar
cal=list(calendar.month_name)
for i in list(calendar.month_name[1:]):
    cal.append(i[:3])

In [24]:
patt1=re.compile("Match ID and/or Full Legal Name",re.IGNORECASE)

# patt2=re.compile("Agreed with analyst recommendation|Agreed with analyst's recommendation|Agreed as per Maker comment|Agreed as per Maker's comment")
patt2=re.compile("(Agreed|agree) (to|with|as per) (analyst(('s)*)|maker(('s)*)) (comment|recommendation|is recommendation)",re.IGNORECASE)
patt3=re.compile("Unknown|false hit\.*|true hit\.*|mismatch\.*|match\.*|non-material|non material\.*|Rationale|Source",re.IGNORECASE)
patt4=re.compile("CSCDD7|CSCDD1|CSCDD4",re.IGNORECASE)
patt5=re.compile("SCC6|No further review is required by analyst|SC/DEL",re.IGNORECASE)
patt6=re.compile("https?://\S+|www\.\S+",re.IGNORECASE)
patt7=re.compile("(%s)"%"|".join(cal)[1:],re.IGNORECASE)
patt8=re.compile("\s*(Mohd|NORAINI|ABDULLAH|BINTI|system|legal|stated|ABD|mahdzir|bin|khalid|siti|hajar|mohd|sallehudin|mohd|saddam|inie|binti|abdullah|noraini|norliza|binti|othman)\s*",re.IGNORECASE)
# patt_11=re.compile("",re.IGNORECASE)

In [25]:
df["V_COMMENTS_PROCESSED"]=df["V_COMMENTS"].apply(lambda x: re.sub("\n"," ",x)) #1
# df["V_COMMENTS_PROCESSED"]=df["V_COMMENTS_PROCESSED"].apply(lambda x: re.sub(patt_11,"nonmaterial",x))
df["V_COMMENTS_PROCESSED"]=df["V_COMMENTS_PROCESSED"].apply(lambda x: re.sub("Name – Name","Name",x))    #2
df["V_COMMENTS_PROCESSED"]=df["V_COMMENTS_PROCESSED"].apply(lambda x: re.sub("Name_Name","Name",x))      #3
df["V_COMMENTS_PROCESSED"]=df["V_COMMENTS_PROCESSED"].apply(lambda x: re.sub("\s*_\s*|\s*-\s*"," ",x)) #4,5
df=df[df["V_COMMENTS_PROCESSED"].str.contains(patt1)==False] #7
df["V_COMMENTS_PROCESSED"]=df["V_COMMENTS_PROCESSED"].apply(lambda x: re.sub(patt2,"",x)) #8
df["V_COMMENTS_PROCESSED"]=df["V_COMMENTS_PROCESSED"].apply(lambda x: re.sub(patt3," ",x))  #  9
df=df[df["V_COMMENTS_PROCESSED"].str.contains(patt4)==False]        # 10
df=df[df["V_COMMENTS_PROCESSED"].str.contains(patt5)==False]       #11
df["V_COMMENTS_PROCESSED"]=df["V_COMMENTS_PROCESSED"].apply(lambda x: re.sub(patt6,"",x)) #15
df["V_COMMENTS_PROCESSED"]=df["V_COMMENTS_PROCESSED"].apply(lambda x: re.sub("[^\w\s\.]|\d"," ",x))
df["V_COMMENTS_PROCESSED"]=df["V_COMMENTS_PROCESSED"].apply(lambda x: re.sub("\d*","",x))    #date, year, accounts 6
df["V_COMMENTS_PROCESSED"]=df["V_COMMENTS_PROCESSED"].apply(lambda x: re.sub(patt7,"",x))    # month 6
df["V_COMMENTS_PROCESSED"]=df["V_COMMENTS_PROCESSED"].apply(lambda x: re.sub(patt8," ",x)) # 17


In [26]:
df["V_COMMENTS_PROCESSED"]=df["V_COMMENTS_PROCESSED"].str.lower()

In [27]:
# import nltk
# nltk.download('punkt')

In [28]:
def remove_stopwords(document):
    "Return the articles after remvoing stopwords"
    article_tokens = word_tokenize(document) 
    filtered_article = [word for word in article_tokens if not word in stop_words] 
    return " ".join(filtered_article)

In [29]:
stop_words = set(stopwords.words('english')) 

In [30]:
#removing stopwords
df.V_COMMENTS_PROCESSED=df.V_COMMENTS_PROCESSED.apply(lambda x: remove_stopwords(x))

In [31]:
df["V_COMMENTS_TOKENS"]=df["V_COMMENTS_PROCESSED"].apply(lambda x: word_tokenize(x))

In [34]:
# for i,row in df.iterrows():
#     if "." in row["V_COMMENTS_TOKENS"]:
#         row["V_COMMENTS_TOKENS"].remove(".")

In [ ]:

df_features["Anchor_2"]=df_features["Anchor_2"].apply(lambda x: remove_stopwords(x))
# df_features["Anchor_Tokens"]=df_features["Anchor_2"].apply(lambda x:word_tokenize(x) )

In [36]:
# l1=df["V_COMMENTS_TOKENS"].to_list()
# l2=df_features["Anchor_Tokens"].to_list()

In [32]:
df["V_COMMENTS_3"]=df["V_COMMENTS_PROCESSED"].apply(lambda x: sent_tokenize(x))


In [33]:
df["V_COMMENTS_3"]

0                                         [id number nric]
1                                [full name sound similar]
2                                         [id number nric]
3                                [full name sound similar]
4                                         [date birth dob]
                               ...                        
91980    [occupation customer work agricultural forestr...
91981    [occupation customer work agricultural forestr...
91982    [occupation customer work agricultural forestr...
91983    [occupation customer work agricultural forestr...
91984    [watchperson sentenced shizuishan municipal in...
Name: V_COMMENTS_3, Length: 91685, dtype: object

In [34]:
df["V_COMMENTS_3"]=df["V_COMMENTS_3"].apply(lambda x:[i.replace(".","") for i in x])

In [35]:
df[["V_COMMENTS","V_COMMENTS_3"]]

,V_COMMENTS,V_COMMENTS_3
0,Mismatch ID number_NRIC,[id number nric]
1,Mismatch Full Legal Name – Name Does Not Sound...,[full name sound similar]
2,Mismatch ID number_NRIC\n,[id number nric]
3,Mismatch Full Legal Name_Name Does Not Sound S...,[full name sound similar]
4,Mismatch Date of Birth (DOB),[date birth dob]
...,...,...
91980,Mismatch Occupation - Customer work as AGRICUL...,[occupation customer work agricultural forestr...
91981,Mismatch Occupation - Customer work as AGRICUL...,[occupation customer work agricultural forestr...
91982,Mismatch Occupation - Customer work as AGRICUL...,[occupation customer work agricultural forestr...
91983,Mismatch Occupation - Customer work as AGRICUL...,[occupation customer work agricultural forestr...


In [41]:
# ind1=df[df["V_COMMENTS_3"].apply(lambda x: np.any([len(i)>50 for i in x]))][["V_COMMENTS_3","V_COMMENTS"]].index

In [42]:
# Sentence bert

In [43]:
# pip install -U sentence-transformers

In [59]:
# Load BERT model and tokenizer
from sentence_transformers import SentenceTransformer, util

# Load a pre-trained BERT model for sentence embeddings

model = model = SentenceTransformer("all-MiniLM-L6-v2")

In [45]:
# Example word and sentence
word = "A human"
sentence = "A human is the most destructive species."

# Obtain embeddings for the word and the sentence
word_embedding = model.encode(word, convert_to_tensor=True)
sentence_embedding = model.encode(sentence, convert_to_tensor=True)

# Calculate cosine similarity between the word and the sentence
similarity_score = util.pytorch_cos_sim(word_embedding, sentence_embedding).item()

print(f"Similarity Score: {similarity_score}")

Similarity Score: 0.3741147816181183


In [46]:
df_features["Feat_vect"]=df_features["Anchor_2"].apply(lambda x: model.encode(x, convert_to_tensor=True))

In [47]:
df_samp=df.loc[:10000]

In [36]:
df_features["Anchor_2"]

0                            word count difference
1                        name structure difference
2                     name does not sound similiar
3                native name charachter difference
5                              date of birth  dob 
                          ...                     
430    customer transaction number of transactions
431                    customer transaction amount
432            customer transaction online offline
433                    customer joint account flag
434               customer last transaction amount
Name: Anchor_2, Length: 428, dtype: object

In [48]:
# df_samp=pd.concat([df_samp,df.tail(100)],axis=0,join="outer")
# 

In [49]:
# df_samp.reset_index(inplace=True)

In [50]:
df_samp["Comm_vec"]=df_samp["V_COMMENTS_3"].apply(lambda x: [model.encode(sent, convert_to_tensor=True) for sent in x])

In [51]:
df_samp["Comm_vec"].head(),df_features["Feat_vect"].head()

(0    [[tensor(-0.1136), tensor(0.0560), tensor(-0.1...
 1    [[tensor(-0.0589), tensor(-0.0617), tensor(-0....
 2    [[tensor(-0.1136), tensor(0.0560), tensor(-0.1...
 3    [[tensor(-0.0589), tensor(-0.0617), tensor(-0....
 4    [[tensor(-0.0567), tensor(0.0560), tensor(-0.0...
 Name: Comm_vec, dtype: object,
 0    [tensor(0.0634), tensor(-0.0251), tensor(-0.01...
 1    [tensor(-0.0765), tensor(-0.0207), tensor(-0.0...
 2    [tensor(-0.0402), tensor(-0.0885), tensor(-0.0...
 3    [tensor(-0.0212), tensor(-0.0580), tensor(-0.0...
 5    [tensor(-0.0567), tensor(0.0560), tensor(-0.00...
 Name: Feat_vect, dtype: object)

In [53]:
df_samp["Features_present_cosine"]=""
def f(x,y):
      l=[]
      for sent_vec,sent in zip(x,y):
          l.append(sorted([[util.pytorch_cos_sim( sent_vec,feat).item(),feat_name, sent] for feat,feat_name in zip(df_features["Feat_vect"],df_features["Anchor_2"]) if util.pytorch_cos_sim( sent_vec,feat).item() >0.5 ],key=lambda x: x[0],reverse=True) )

      return l
for i,vecs,comm in zip(df_samp.index[:1000],df_samp["Comm_vec"].loc[:999],df_samp["V_COMMENTS_3"].loc[:999]):

    df_samp["Features_present_cosine"].loc[i]=f(vecs,comm)
# df_samp[df_samp.loc[:,"Vect"].apply(lambda x:np.array(x).shape==(9,))]["Vect"][24230]
# df_samp["Vect_copy"]=df_samp["Vect"].apply(lambda x:[np.zeros((1,50) ) if np.any(pd.isna(i)) else i for i in x  ])

In [54]:
df_samp["Features_present_cosine"]=df_samp["Features_present_cosine"].apply(lambda x: [i for i in x if i!=[]])

In [ ]:
# df_samp["Features_present_cosine_2"]=df_samp["Features_present_cosine"].apply(lambda x: x[0] if len(x)>1 else x)

In [55]:

df_samp["Features_present_cosine"]=df_samp["Features_present_cosine"].apply(lambda x: [ j for i in x for j in i])

In [57]:
# df_samp["Features_present_cosine_top_5"]

In [58]:
df_samp["Features_present_cosine_top_5"]=df_samp["Features_present_cosine"].apply(lambda x: sorted(x,key=lambda x:x[0],reverse=True))

In [59]:
df_samp["Features_present_cosine_actual_anchors"]=df_samp["Features_present_cosine_top_5"].apply(lambda x: [i[1] for i in x])

In [60]:
df_samp["Features_present_cosine_actual_anchors"]=df_samp["Features_present_cosine_actual_anchors"].apply(lambda x: list(set(x)))

In [61]:
df_samp["Feat_Categories"]=""

In [62]:
categories=[]
for i in df_features[["Anchor_2","Categories"]].values:
    categories.append(list(i))

In [63]:
categories

[['word count difference', 'Full Legal Name - Word Count'],
 ['name structure difference', 'Full Legal Name - Name Structure'],
 ['name sound similiar', 'Full Legal Name - Name Does Not Sound Similar'],
 ['native name charachter difference', 'Full Legal Name - Native Name'],
 ['date birth dob', 'Date of Birth (DOB)'],
 ['year birth yob age age date', 'Age/ Year of Birth (YOB)'],
 ['match place birth', 'Place of Birth'],
 ['id number nric', 'Identification Number - NRIC'],
 ['id citizenship number', 'Identification Number - Citizenship Number'],
 ['watch person deceased date', 'Deceased Status'],
 ['watch person age deceased', 'Deceased Status'],
 ['watch person deceased flag', 'Deceased Status'],
 ['customer deceased flag', 'Deceased Status'],
 ['customer deceased date', 'Deceased Status'],
 ['gender', 'Gender'],
 ['father name', "Father's Name "],
 ['customer occupation biography profile', 'Biography/ Occupation'],
 ['customer occupation biography profile', 'Biography/ Occupation'],
 

In [64]:
for ind,anch in zip(df_samp.index,df_samp["Features_present_cosine_actual_anchors"]):
    l=[]
    for x in anch:
        l.append([(x,i[1])  for i in categories if x in i][0])
    df_samp["Feat_Categories"].loc[ind]=l

In [65]:
df_samp["Feat_Categories"]

0        [(match aunt id number nric, Related Parties),...
1        [(name sound similiar, Full Legal Name - Name ...
2        [(match aunt id number nric, Related Parties),...
3        [(name sound similiar, Full Legal Name - Name ...
4        [(match father date year birth dob yob, Relate...
                               ...                        
9996                                                    []
9997                                                    []
9998                                                    []
9999                                                    []
10000                                                   []
Name: Feat_Categories, Length: 9878, dtype: object

In [66]:
df_samp["Categories"]=df_samp["Feat_Categories"].apply(lambda x: list(set([i[1] for i in x])))

In [67]:
df_samp["V_COMMENTS_PROCESSED"]


0                                           id number nric
1                                  full name sound similar
2                                           id number nric
3                                  full name sound similar
4                                           date birth dob
                               ...                        
9996     profile . watchperson father babar hussain abi...
9997     profile occupation . watchperson pep member me...
9998     profile occupation . watchperson son pep minis...
9999     profile . watchperson member regional house re...
10000    profile . watchperson member regional house re...
Name: V_COMMENTS_PROCESSED, Length: 9878, dtype: object

In [68]:
related_party_list=['Spouse', 'Mother', 'Father', 'Brother', 'Sister',
       'Son', 'Daughter', 'Son in law', 'Daughter in law',
       'Father in law', 'Mother in law', 'Sister in law',
       'Brother in law', 'Grandparents', 'Grandchildren', 'Uncle', 'Aunt',
       'Cousin', 'Driver', 'Bodyguard', 'Secretaries', 'Business',
       'Ex Husband']
related_party_list=[prt.lower() for prt in related_party_list]

In [69]:
df_samp["Rel_flag"]=df_samp[df_samp["Categories"].apply(lambda x: "Related Parties" in x)]["V_COMMENTS_TOKENS"].apply(lambda x:sum([1 if tok in x else 0 for tok in related_party_list])>0)

In [70]:
df_samp[df_samp["Rel_flag"]==False]["Categories"].apply(lambda x: x.remove("Related Parties"))

0      None
2      None
4      None
6      None
7      None
       ... 
988    None
990    None
993    None
997    None
999    None
Name: Categories, Length: 631, dtype: object

In [71]:
df_samp["Categories"]

0                           [Identification Number - NRIC]
1          [Full Legal Name - Name Does Not Sound Similar]
2                           [Identification Number - NRIC]
3          [Full Legal Name - Name Does Not Sound Similar]
4        [Age/ Year of Birth (YOB), Date of Birth (DOB)...
                               ...                        
9996                                                    []
9997                                                    []
9998                                                    []
9999                                                    []
10000                                                   []
Name: Categories, Length: 9878, dtype: object

In [73]:
df_samp[['V_COMMENTS',"Counts",'V_COMMENTS_PROCESSED', 'V_COMMENTS_3',
        'Features_present_cosine_actual_anchors',"Feat_Categories","Categories"]].to_csv("Sentbert_output.csv")

In [ ]:
df["V_COMMENTS_3"]=df["V_COMMENTS_3"].apply(lambda x:[word_tokenize(sent) for sent in x ])

In [ ]:
df["V_COMMENTS_3"]

In [ ]:
l1=df["V_COMMENTS_3"].apply(lambda x: [ j for i in x for j in i]).to_list()

In [ ]:
l1.extend(l2)

In [ ]:
# l2=[]
# for l in df.loc[ind1,"V_COMMENTS_3"].head():
#     l3=[]
#     for i in l:
#         if len(i)>50:
#               inputs=tokenizer.encode(i,return_tensors='pt', max_length=512, truncation=True)
#               output = model.generate(inputs, min_length=15, max_length=len(i))
#               summary=tokenizer.decode(output[0])
#               l3.append(summary)
#         else:
#               l3.append(i)
#     l2.append(l3)

In [ ]:
model1= gensim.models.Word2Vec(l1, min_count=1,
                                vector_size=50, window=5,sg=0)

In [ ]:
df["Vect"]=df["V_COMMENTS_3"].apply(lambda x:[np.sum(np.array([model1.wv[i] for i in sent]),axis=0)/(len(sent)) for sent in x])

In [ ]:
df["Vect"]

In [ ]:
df_features["Vect 2"]=df_features["Anchor_Tokens"].apply(lambda x:np.sum(np.array([model1.wv[i] for i in x]),axis=0)/(len(x)))

In [ ]:
df_features[["Vect 2","Anchor_Tokens"]]

In [ ]:
df["Vect_copy"]=df["Vect"].apply(lambda x: [ np.zeros((1,50))[0] if np.any(i==np.nan) or np.any(pd.isna(i)) else i for i in x])

In [ ]:
df["Vect_copy"]

In [ ]:
def f(x):
      l=[]
      for sent_vec in x:
          l.append(sorted([(cosine_similarity([sent_vec],[feat])[0][0],feat_name )for feat,feat_name in zip(df_features["Vect 2"],df_features["Anchor_Topic_Tokens"])],key=lambda x: x[0],reverse=True)[:3] )

      return l
for i,vecs in zip(df.index,df["Vect_copy"]):

    df["Features_present"].loc[i]=f(vecs)
# df[df.loc[:,"Vect"].apply(lambda x:np.array(x).shape==(9,))]["Vect"][24230]
# df["Vect_copy"]=df["Vect"].apply(lambda x:[np.zeros((1,50) ) if np.any(pd.isna(i)) else i for i in x  ])

In [ ]:
df_samp["Features_present_cosine"]=df_samp["Features_present_cosine"].apply(lambda x: [ j for i in x for j in i])

In [ ]:
df_samp["Features_present_cosine_top_5"]=df_samp["Features_present_cosine"].apply(lambda x: sorted(x,key=lambda x:x[0],reverse=True)[:5])

In [ ]:
### The end
# df_samp["Features_present_cosine_top_5"]=df_samp["Features_present_cosine"].apply(lambda x: x[:5])

In [ ]:
df_samp["Features_present_cosine_actual_anchors"]=df_samp["Features_present_cosine_top_5"].apply(lambda x: [i[1] for i in x]).apply(lambda x: [ " ".join(i) for i in x])

In [ ]:
df_samp["Features_present_cosine_actual_anchors"]=df_samp["Features_present_cosine_actual_anchors"].apply(lambda x: list(set(x)))

In [ ]:
df_samp["Feat_Categories"]=""

In [ ]:
categories=[]
for i in df_features[["Anchor_2","Categories"]].values:
    categories.append(list(i))

In [ ]:
categories

In [ ]:
for ind,anch in zip(df_samp.index,df_samp["Features_present_cosine_actual_anchors"]):
    l=[]
    for x in anch:
        l.append([(x,i[1])  for i in categories if x in i][0])
    df_samp["Feat_Categories"].loc[ind]=l

In [ ]:
df_samp["Feat_Categories"]

In [ ]:
df_samp[["V_COMMENTS","Counts","Match_Mismatch","V_COMMENTS_3","Features_present_cosine_top_5","Features_present_cosine_actual_anchors","Feat_Categories","Categories"]].to_csv("Feature_extraction_nov_Jaccard_sim_4.csv")

In [ ]:
df_samp["Categories"]=df_samp["Feat_Categories"].apply(lambda x: list(set([i[1] for i in x])))

In [ ]:
# df_samp[["Categories","Counts"]].apply(lambda x: [ for i in x],axis=1)

In [ ]:
# df_samp[""]

In [ ]:
df_features["Categories"].unique()


In [ ]:
df_stats=pd.DataFrame({"Categories":df_features["Categories"].unique()[:-1],})

In [ ]:
df_stats["Counts"]=""

In [ ]:
for i,rows in df_stats.iterrows():
    df_stats.loc[i,"Counts"]=df_samp[df_samp["Categories"].apply(lambda x: rows["Categories"] in x)]["Counts"].sum()


In [ ]:
max=df_samp["Counts"].sum()
df_stats["Percentage"]=df_stats["Counts"].apply(lambda x: x/ max)

In [ ]:
df_stats.to_csv("Categories_stats.csv")

### Ollama

In [10]:
# ollama embeddings
from langchain_community.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings

In [17]:
# Clone llama.cpp repo
git clone https://github.com/ggerganov/llama.cpp.git

# Enter llama.cpp project dir
cd llama.cpp

# On Mac, using Metal allows the computation to be executed on the GPU for Apple devices
LLAMA_METAL=1 make

SyntaxError: invalid syntax (916910490.py, line 2)

### Ollama Embeddings

In [68]:
import os
import langchain
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
ollama_emb = OllamaEmbeddings()

ollama_emb_2 = OllamaEmbeddings( model="llama2")
ollama_emb_3 = OllamaEmbeddings(model="llama3")
#from dotenv import load_dotenv
#load_dotenv()

 
#api_key=os.getenv("OPENAI_KEY",None)

# from langchain.llms import OpenAI
 
# llm = OpenAI(temperature=0.9,openai_api_key='OPENAI_KEY')

In [11]:
embeddings = OllamaEmbeddings()
text = "This is a test document."
query_result = embeddings.embed_query(text)
query_result[:5]

[-0.10140955448150635,
 0.011419410817325115,
 0.1745792180299759,
 0.1644006073474884,
 0.20826418697834015]

In [30]:
import torch
query_result =[ torch.tensor(embeddings.embed_query(text)) for text in df_features["Anchor_2"]]

In [54]:
df_features["Feat_vect"]=query_result

In [50]:
df_samp=df.loc[:100]

In [53]:
df_samp["Comm_vec_ollama"]=df_samp["V_COMMENTS_3"].apply(lambda x: [torch.tensor(embeddings.embed_query(sent)) for sent in x])

In [61]:
df_samp["Features_present_cosine_ollama"]=""
def f(x,y):
      l=[]
      for sent_vec,sent in zip(x,y):
          l.append(sorted([[util.pytorch_cos_sim( sent_vec,feat).item(),feat_name, sent] for feat,feat_name in zip(df_features["Feat_vect"],df_features["Anchor_2"]) if util.pytorch_cos_sim( sent_vec,feat).item() >0.5 ],key=lambda x: x[0],reverse=True) )

      return l
for i,vecs,comm in zip(df_samp.index[:100],df_samp["Comm_vec_ollama"].loc[:99],df_samp["V_COMMENTS_3"].loc[:99]):

    df_samp["Features_present_cosine_ollama"].loc[i]=f(vecs,comm)
# df_samp[df_samp.loc[:,"Vect"].apply(lambda x:np.array(x).shape==(9,))]["Vect"][24230]
# df_samp["Vect_copy"]=df_samp["Vect"].apply(lambda x:[np.zeros((1,50) ) if np.any(pd.isna(i)) else i for i in x  ])

In [62]:
df_samp["Features_present_cosine_ollama"]=df_samp["Features_present_cosine_ollama"].apply(lambda x: [i for i in x if i!=[]])

In [ ]:
# df_samp["Features_present_cosine_2"]=df_samp["Features_present_cosine"].apply(lambda x: x[0] if len(x)>1 else x)

In [64]:

df_samp["Features_present_cosine_ollama"]=df_samp["Features_present_cosine_ollama"].apply(lambda x: [ j for i in x for j in i])

In [57]:
# df_samp["Features_present_cosine_top_5"]

In [65]:
df_samp["Features_present_cosine_top_ollama"]=df_samp["Features_present_cosine_ollama"].apply(lambda x: sorted(x,key=lambda x:x[0],reverse=True))

In [66]:
df_samp["Features_present_cosine_actual_anchors_ollama"]=df_samp["Features_present_cosine_top_ollama"].apply(lambda x: [i[1] for i in x])

In [67]:
df_samp

,V_COMMENTS,N_PARTY_WATCHLIST_MATCH_ID,Counts,Match_Mismatch,V_COMMENTS_PROCESSED,V_COMMENTS_TOKENS,V_COMMENTS_3,Comm_vec_ollama,Features_present_cosine_ollama,Features_present_cosine_top_ollama,Features_present_cosine_actual_anchors_ollama
0,Mismatch ID number_NRIC,"[6147381, 6003260, 5488573, 6013209, 6007897, ...",20859,[Mismatch],id number nric,"[id, number, nric]",[id number nric],"[[tensor(-2.7573), tensor(-0.2343), tensor(-2....","[[0.9999998807907104, id number nric, id numbe...","[[0.9999998807907104, id number nric, id numbe...","[id number nric, match uncle id number nric,..."
1,Mismatch Full Legal Name – Name Does Not Sound...,"[6081737, 4832938, 7491187, 6227658, 5401157, ...",13039,[Mismatch],full name sound similar,"[full, name, sound, similar]",[full name sound similar],"[[tensor(-0.3221), tensor(-1.1757), tensor(2.6...","[[0.6600239872932434, match son in law name, ...","[[0.6600239872932434, match son in law name, ...","[match son in law name, match sister name, m..."
2,Mismatch ID number_NRIC\n,"[5129297, 5086069, 6260885, 5277278, 4821963, ...",8666,[Mismatch],id number nric,"[id, number, nric]",[id number nric],"[[tensor(-2.7573), tensor(-0.2343), tensor(-2....","[[0.9999998807907104, id number nric, id numbe...","[[0.9999998807907104, id number nric, id numbe...","[id number nric, match uncle id number nric,..."
3,Mismatch Full Legal Name_Name Does Not Sound S...,"[6190699, 5155337, 6121364, 5155334, 5155339, ...",6680,[Mismatch],full name sound similar,"[full, name, sound, similar]",[full name sound similar],"[[tensor(-0.3221), tensor(-1.1757), tensor(2.6...","[[0.6600239872932434, match son in law name, ...","[[0.6600239872932434, match son in law name, ...","[match son in law name, match sister name, m..."
4,Mismatch Date of Birth (DOB),"[4801383, 4887727, 5254984, 4806610, 6031120, ...",6288,[Mismatch],date birth dob,"[date, birth, dob]",[date birth dob],"[[tensor(1.1900), tensor(-0.6665), tensor(0.68...","[[0.7642505764961243, customer cif creation da...","[[0.7642505764961243, customer cif creation da...","[customer cif creation date, watch person driv..."
...,...,...,...,...,...,...,...,...,...,...,...
96,Mismatch Year of Birth (YOB) Watchperson YOB a...,"[7703770, 7701952, 7702281, 7704132, 7702672, ...",291,[Mismatch],year birth yob watchperson yob per worldcheck ...,"[year, birth, yob, watchperson, yob, per, worl...",[year birth yob watchperson yob per worldcheck...,"[[tensor(-0.8344), tensor(0.2042), tensor(-0.1...","[[0.7229762077331543, customer son in law tran...","[[0.7229762077331543, customer son in law tran...","[customer son in law transaction location, cus..."
97,False Hit-Mismatch ID number _ NRIC,"[4811907, 5226796, 6065103, 6285085, 5161323, ...",277,"[False Hit, Mismatch]",id number nric,"[id, number, nric]",[id number nric],"[[tensor(-2.7573), tensor(-0.2343), tensor(-2....","[[0.9999998807907104, id number nric, id numbe...","[[0.9999998807907104, id number nric, id numbe...","[id number nric, match uncle id number nric,..."
98,Mismatch Biography/Occupation – Customer worke...,"[7500615, 7500616, 7511543, 7500617, 7511544, ...",268,[Mismatch],biography occupation customer worked mining co...,"[biography, occupation, customer, worked, mini...",[biography occupation customer worked mining c...,"[[tensor(1.3293), tensor(-0.3780), tensor(3.52...","[[0.7316364645957947, watch person job work lo...","[[0.748814582824707, watch person sister pep m...","[watch person sister pep more than 7 years , w..."
99,Mismatch ID number_NRIC.\n,"[6043578, 4712638, 6043766, 6044266, 6043684, ...",258,[Mismatch],id number nric .,"[id, number, nric, .]",[id number nric ],"[[tensor(-1.9529), tensor(-0.5195), tensor(-1....","[[0.7575944662094116, match son in law passpo...","[[0.7575944662094116, match son in law passpo...","[match son in law passport number , match un..."


In [69]:
df_samp["Feat_Categories"]=""

In [70]:
categories=[]
for i in df_features[["Anchor_2","Categories"]].values:
    categories.append(list(i))

In [71]:
categories

[['word count difference', 'Full Legal Name - Word Count'],
 ['name structure difference', 'Full Legal Name - Name Structure'],
 ['name does not sound similiar',
  'Full Legal Name - Name Does Not Sound Similar'],
 ['native name charachter difference', 'Full Legal Name - Native Name'],
 ['date of birth  dob ', 'Date of Birth (DOB)'],
 ['year of birth  yob   age and age of date', 'Age/ Year of Birth (YOB)'],
 ['match  place of birth', 'Place of Birth'],
 ['id number nric', 'Identification Number - NRIC'],
 ['id citizenship number', 'Identification Number - Citizenship Number'],
 ['watch person deceased date ', 'Deceased Status'],
 ['watch person age as of deceased', 'Deceased Status'],
 ['watch person deceased flag', 'Deceased Status'],
 ['customer deceased flag', 'Deceased Status'],
 ['customer deceased date', 'Deceased Status'],
 ['gender', 'Gender'],
 ['father name', "Father's Name "],
 ['customer occupation biography profile', 'Biography/ Occupation'],
 ['customer occupation biograp

In [72]:
for ind,anch in zip(df_samp.index,df_samp["Features_present_cosine_actual_anchors_ollama"]):
    l=[]
    for x in anch:
        l.append([(x,i[1])  for i in categories if x in i][0])
    df_samp["Feat_Categories"].loc[ind]=l

In [73]:
df_samp["Feat_Categories"]


0      [(id number nric, Identification Number - NRIC...
1      [(match  son in law name, Related Parties), (m...
2      [(id number nric, Identification Number - NRIC...
3      [(match  son in law name, Related Parties), (m...
4      [(customer cif creation date, UNKOWN/Miscellan...
                             ...                        
96     [(customer son in law transaction location, Re...
97     [(id number nric, Identification Number - NRIC...
98     [(watch person sister pep more than 7 years , ...
99     [(match  son in law passport number , Related ...
100                                                   []
Name: Feat_Categories, Length: 99, dtype: object

In [75]:
df_samp["Categories"]=df_samp["Feat_Categories"].apply(lambda x: list(set([i[1] for i in x])))

In [76]:
df_samp[["V_COMMENTS","Counts","Match_Mismatch","V_COMMENTS_3","Features_present_cosine_ollama","Features_present_cosine_top_ollama","Features_present_cosine_actual_anchors_ollama","Feat_Categories","Categories"]].to_csv("Feature_extraction_ollama_results.csv")

In [ ]:
df_samp["Categories"]=df_samp["Feat_Categories"].apply(lambda x: list(set([i[1] for i in x])))